## Answering Business Questions using complex SQL queries

In this project I will answer business questions using complex queries in SQL about the "Chinook" database. 
Chinook is a fictional music store. The database can be found in this repository as well as the schema. 

I will be running SQL queries in Jupyter Notebook (which itself is running in Python 3). 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook-unmodified.db

Here, I connect SQLLite to my database.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook-unmodified.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


I ensure that my database looks good according to the schema and I can see what it consists of. 
I will run a couple random queries to get familiar with what the data in my db looks like.

Random query 1:

In [3]:
%%sql
SELECT e.last_name || " " || e.first_name support_rep,
       c.last_name || " " || c.first_name customer,
       c.email customer_email
FROM employee e
LEFT JOIN customer c ON e.employee_id = c.support_rep_id
ORDER BY customer DESC
LIMIT 10;

 * sqlite:///chinook-unmodified.db
Done.


support_rep,customer,customer_email
Peacock Jane,Zimmermann Fynn,fzimmermann@yahoo.de
Park Margaret,Wójcik Stanisław,stanisław.wójcik@wp.pl
Park Margaret,Wichterlová František,frantisekw@jetbrains.com
Johnson Steve,Van der Berg Johannes,johavanderberg@yahoo.nl
Peacock Jane,Tremblay François,ftremblay@gmail.com
Park Margaret,Taylor Mark,mark.taylor@yahoo.au
Peacock Jane,Sullivan Ellie,ellie.sullivan@shaw.ca
Johnson Steve,Stevens Victor,vstevens@yahoo.com
Peacock Jane,Srivastava Puja,puja_srivastava@yahoo.in
Johnson Steve,Smith Jack,jacksmith@microsoft.com


Random query 2:

In [4]:
%%sql
SELECT i.billing_address billing_addy,
       i.customer_id custo_id,
       pt.playlist_id
       
FROM invoice i
INNER JOIN invoice_line il ON i.invoice_id=il.invoice_line_id
INNER JOIN playlist_track pt ON il.track_id = pt.track_id
WHERE custo_id > 50
ORDER BY billing_addy
LIMIT 10;

 * sqlite:///chinook-unmodified.db
Done.


billing_addy,custo_id,playlist_id
110 Raeburn Pl,54,1
110 Raeburn Pl,54,8
110 Raeburn Pl,54,1
110 Raeburn Pl,54,8
110 Raeburn Pl,54,1
110 Raeburn Pl,54,8
110 Raeburn Pl,54,1
110 Raeburn Pl,54,8
110 Raeburn Pl,54,1
110 Raeburn Pl,54,8


## Business Question 1: Finding which genres sell the most tracks in the USA

"The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

Artist Name 	         Genre
Regal 	                Hip-Hop
Red Tone 	            Punk
Meteor and the Girls 	Pop
Slim Jim Bites 	        Blues

Write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store."

In [5]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )
    
SELECT
    g.name genre,
    count(uts.invoice_line_id) tracks_sold,
    cast(count(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///chinook-unmodified.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


## Business Question 2: Find the amount of sales assigned to Sales Agents

"Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. Analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others."

In [6]:
%%sql

WITH customer_support_rep_sales AS
    (
     SELECT
         i.customer_id,
         c.support_rep_id,
         SUM(i.total) total
     FROM invoice i
     INNER JOIN customer c ON i.customer_id = c.customer_id
     GROUP BY 1,2
    )

SELECT
    e.first_name || " " || e.last_name employee,
    e.hire_date,
    SUM(csrs.total) total_sales
FROM customer_support_rep_sales csrs
INNER JOIN employee e ON e.employee_id = csrs.support_rep_id
GROUP BY 1;

 * sqlite:///chinook-unmodified.db
Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,1393.92


## Business Question 3: Analyze sales data for customers from different countries

Calculate data, for each country, on the:

    total number of customers
    total value of sales
    average value of sales per customer
    average order value


In [7]:
%%sql
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///chinook-unmodified.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


## Business Question 4: Categorize invoices and calculate statistics

Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:

    Number of invoices
    Percentage of invoices


In [8]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )
SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook-unmodified.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674
